In [1]:
import os
import rasterio
from rasterio.windows import Window
import pandas as pd
from tqdm import tqdm

def generate_tile_metadata(tif_path, tile_size=1024):
    img_name = os.path.splitext(os.path.basename(tif_path))[0]
    metadata = []

    with rasterio.open(tif_path) as src:
        for y in range(0, src.height, tile_size):
            for x in range(0, src.width, tile_size):
                width = min(tile_size, src.width - x)
                height = min(tile_size, src.height - y)
                tile_name = f"{img_name}_tile_{x}_{y}.tif"
                metadata.append({
                    "tile_name": tile_name,
                    "tile_x": x,
                    "tile_y": y,
                    "tile_width": width,
                    "tile_height": height
                })

    return metadata

def main():
    tif_dir = "/shared/data/climateplus2025/CapeTown_Image_2023_original"
    output_csv = "/shared/data/climateplus2025/YOLO+U-Net_Prediction_3images_updated_head_to_head_comparision_0722/tile_metadata_for_reconstruction.csv"
    tile_size = 1024

    selected_image_names = [
        '2023_RGB_8cm_W25C_16',
        '2023_RGB_8cm_W24A_17',
        '2023_RGB_8cm_W57B_8'
    ]

    # Map .tif file paths
    tif_map = {}
    for root, dirs, files in os.walk(tif_dir):
        for f in files:
            if f.endswith(".tif"):
                name = os.path.splitext(f)[0]
                tif_map[name] = os.path.join(root, f)

    # Process
    all_metadata = []
    for name in tqdm(selected_image_names, desc="Generating tile metadata"):
        if name in tif_map:
            meta = generate_tile_metadata(tif_map[name], tile_size)
            all_metadata.extend(meta)
        else:
            print(f"[!] Missing: {name}.tif")

    # Save
    df = pd.DataFrame(all_metadata)
    df.to_csv(output_csv, index=False)
    print(f"Metadata saved to {output_csv}")

if __name__ == "__main__":
    main()

Generating tile metadata: 100%|██████████| 3/3 [00:00<00:00, 95.42it/s]

Metadata saved to /shared/data/climateplus2025/YOLO+U-Net_Prediction_3images_updated_head_to_head_comparision_0722/tile_metadata_for_reconstruction.csv
